In [1]:
import numpy as np
import scipy as sp
from numpy import linalg as nla
import matplotlib.pyplot as plt

In [2]:
def model(r):
    x=r[0]
    y=r[1]
    z=r[2]
    rho=0.3224
    delta=0.2625
    a=6.8927
    k=0.4032
    c=2.3952
    x1=1.0*(rho*delta*(x*x-a*x)+x*(x+y+c-c*np.tanh(x+z)))
    y1=-1.0*rho*delta*(a*y+x*x)
    z1=delta*(k-z-0.5*x)
    return (np.array([x1,y1,z1]))

In [3]:
def jac(r,g1):
    p=0.3224
    d=0.2625
    a=6.8927
    c=2.3952
    x=r[0]
    y=r[1]
    z=r[2]
    M=np.array([[-c*np.tanh(x + z) + c + d*p*(-a + 2*x) + x*(-c*(1 - np.tanh(x + z)**2) + 1) + x + y,x,-c*x*(1 - np.tanh(x + z)**2)],[-2*x*p*d,-p*d*a,0],[-d*0.5,0,-d]])
    res=np.matmul(M,g1)
    return(res)

In [8]:
g1=np.array([1,0,0])
g2=np.array([0,1,0])
g3=np.array([0,0,1])
tmax=400
h=0.01
t=0
x=-3
y=0
z=0
r=np.array([x,y,z])
talp=0
talp2=0
talp3=0
valt=[]
amp1=[]
amp2=[]
amp3=[]
valx=[]
tr=0
p=0.3224
d=0.2625
a=6.8927
c=2.3952
while (t<=tmax):
    xt=r[0]
    yt=r[1]
    zt=r[2]
    Ma=np.array([[-c*np.tanh(xt + zt) + c + d*p*(-a + 2*xt) + xt*(-c*(1 - np.tanh(xt + zt)**2) + 1) + xt + yt,xt,-c*xt*(1 - np.tanh(xt + zt)**2)],[-2*xt*p*d,-p*d*a,0],[-d*0.5,0,-d]])
    trace=Ma.trace()
    tr=tr+trace
    l1=jac(r,g1)
    l2=jac(r+0.5*h*l1,g1)
    l3=jac(r+0.5*h*l2,g1)
    l4=jac(r+h*l3,g1)
    
    g1=g1+(1.0/6)*h*(l1+2*l2+2*l3+l4)
    norm=np.sqrt(g1[0]**2+g1[1]**2+g1[2]**2)
    alp=np.log(norm)
    talp=talp+alp
    g1=g1/norm
    
    interm=(g2-g1*(np.dot(g2,g1)))
    ninterm=np.sqrt(interm[0]**2+interm[1]**2+interm[2]**2)
    g2=interm/ninterm
    alp2=np.log(ninterm)
    talp2=talp2+alp2
    
    interm2=(g3-g2*(np.dot(g3,g2))-g1*(np.dot(g3,g1)))
    ninterm2=np.sqrt(interm2[0]**2+interm2[1]**2+interm2[2]**2)
    g3=interm2/ninterm2
    alp3=np.log(ninterm2)
    talp3=talp3+alp3
    
    k1=model(r)
    k2=model(r+0.5*h*k1)
    k3=model(r+0.5*h*k2)
    k4=model(r+k3*h)
    t=t+h
    valt.append(t)
    amp1.append(talp/t)
    amp2.append(talp2/t)
    r=r+(1.0/6)*h*(k1+2*k2+2*k3+k4)
    valx.append(r[0])
print (talp/tmax)
print (talp2/tmax)
print (talp3/tmax)
print(tr/(tmax/h))

-0.0003852073081710089
-0.002725957920967404
-0.004261847686090479
-0.4815007857449318


In [9]:
xt=r[0]
yt=r[1]
zt=r[2]
print(xt,yt,zt)

-0.5353918408561577 -0.24375221991564877 1.204177011409551


In [10]:
p=0.3224
d=0.2625
a=6.8927
c=2.3952
Ma=np.array([[-c*np.tanh(xt + zt) + c + d*p*(-a + 2*xt) + xt*(-c*(1 - np.tanh(xt + zt)**2) + 1) + xt + yt,xt,-c*xt*(1 - np.tanh(xt + zt)**2)],[-2*xt*p*d,-p*d*a,0],[-d*0.5,0,-d]])
print(Ma)
Ma.trace()

[[-0.14777352 -0.53539184  0.84474042]
 [ 0.09062042 -0.5833292   0.        ]
 [-0.13125     0.         -0.2625    ]]


-0.9936027247434696

In [ ]:
fig, ax1 = plt.subplots()

color = 'tab:blue'
ax1.set_xlabel('time (s)')
ax1.set_ylabel('exponent', color=color)
ax1.plot(valt, amp1, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  

color = 'tab:red'
ax2.set_ylabel('x', color=color)  
ax2.plot(valt, valx, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  
plt.show()

In [ ]:
plt.plot(valt,amp2,label='exp2')
plt.plot(valt,amp1,label='exp1')
plt.xlabel('time')
plt.legend()
plt.show()